Cargamos los datos procesados y preparamos para Surpirsa usando train_df

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity

# 1. Carga de datos
processed_dir = 'data/processed'
train = pd.read_csv(os.path.join(processed_dir, 'train.csv'))
test  = pd.read_csv(os.path.join(processed_dir, 'test.csv'))

# 2. Matriz usuario×libro
#    Asumimos user_id y book_id estén indexados desde 1 y contiguos
num_users = train['user_id'].max()
num_items = train['book_id'].max()

R = np.zeros((num_users, num_items), dtype=np.float32)
for _, row in train.iterrows():
    R[int(row.user_id)-1, int(row.book_id)-1] = row.rating

# 3. Medias de cada usuario
user_means = np.true_divide(R.sum(axis=1), (R!=0).sum(axis=1))
# Evitar división por cero
user_means = np.where(np.isnan(user_means), 0, user_means)

# 4. Matriz de desviaciones centradas
D = np.zeros_like(R)
mask = (R != 0)
D[mask] = R[mask] - user_means[mask.argmax(axis=1)][:, None][mask]

# 5. Similitud coseno entre usuarios
#    Añadimos un pequeño eps para estabilidad
sim = cosine_similarity(D + 1e-9)

# 6. Función de predicción KNN
def predict_knn(u, i, k=20):
    """
    Predice rating del usuario u por el ítem i (ambos en base 0).
    """
    # vecinos que han valorado el ítem i
    voters = np.where(R[:, i] != 0)[0]
    if len(voters) == 0:
        return user_means[u]
    # similitudes y votantes distintos de u
    sims = sim[u, voters]
    # coger los k vecinos más similares
    idx = np.argsort(sims)[-k:]
    top_sims = sims[idx]
    top_devs = D[voters[idx], i]
    if top_sims.sum() == 0:
        return user_means[u]
    pred = user_means[u] + np.dot(top_sims, top_devs) / top_sims.sum()
    # recortar al rango [1,5]
    return min(5, max(1, pred))

# 7. Predicción y evaluación
y_true, y_pred = [], []
for _, row in test.iterrows():
    u = int(row.user_id) - 1
    i = int(row.book_id) - 1
    y_true.append(row.rating)
    y_pred.append(predict_knn(u, i, k=20))

rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae  = mean_absolute_error(y_true, y_pred)

print(f"KNN colaborativo manual → RMSE: {rmse:.4f}, MAE: {mae:.4f}")
